This is where I am experimenting with how to call and download the data from datahub's Covid-19 database

The location of the data site is https://datahub.io/core/covid-19#data
The files are in both JSON and CSV file format

US data is located at /core/covid-19/r/us_confirmed.cvs (or .json)
US data is located at /core/covid-19/r/us_deaths.cvs (or .json)

In [1]:
import requests
import bs4
import os
import csv

In [2]:
file_name = "covid-19-data.html"
base_url = "https://datahub.io/core/covid-19#data"

In [8]:
## Code to search the datahub website to find and download the latest data.
## This can of course be addressed manually but for practice this code would 
## help me search for the link to download the data

## This can of course be addressed manually but for practice this code would 
## help me search for the link to download the data

if os.path.isfile(file_name):
    print(f"Using locally cached data from {file_name}")
    with open(file_name) as file:
        html_text = file.read()
else:
    print(f"Loading data from {base_url}")
    req = requests.get(base_url)
    req.raise_for_status()
    html_text = req.text
    with open(file_name, "w") as file:
        file.write(html_text)
res = bs4.BeautifulSoup(html_text, "lxml")

Loading data from https://datahub.io/core/covid-19#data


UnicodeEncodeError: 'charmap' codec can't encode characters in position 90866-90880: character maps to <undefined>

In [53]:
req = requests.get(base_url)

In [54]:
req.raise_for_status()

In [64]:
html_text = req.text

In [57]:
req.encoding

'utf-8'

In [65]:
html_text=req.content

In [68]:
html_text.decode("utf-8")

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="utf-8">\n  <title>\n    \nNovel Coronavirus 2019 - Dataset\n\n    - DataHub - Frictionless Data\n  </title>\n  <meta name="author" content="Datopian">\n  <meta name="keywords" content="data,open data,datasets,public data,data package,reference data,csv,json,excel">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">\n  <!-- Twitter card -->\n  <meta name="twitter:card" content="summary" />\n  <meta name="twitter:site" content="@datahubio" />\n  <meta name="twitter:title" content="Novel Coronavirus 2019" />\n  <meta name="twitter:image" content="https://datahub.io/core/covid-19/view/0.png" />\n\n  <!-- Open Graph -->\n  <meta property="og:title" content="Novel Coronavirus 2019" />\n  <meta property="og:type" content="website" />\n  <meta property="og:site_name" content="DataHub" />\n  <meta property="og:image" content="https://datahub.io/core/covid-19/view/0.png" />\n 

In [70]:
f=open (file_name, "w")
f.write(html_text.decode())
f.close

UnicodeEncodeError: 'charmap' codec can't encode characters in position 90872-90886: character maps to <undefined>

In [27]:
from IPython.display import IFrame

IFrame(src= file_name, width=700, height=600)

In [9]:
table = res.find('table',{"class":["table table-condensed table-striped resource-listing"]})
table

<table class="table table-condensed table-striped resource-listing">
<thead>
<th class="col-xs-2">File</th>
<th class="col-xs-4">Description</th>
<th class="col-xs-1">Size</th>
<th class="col-xs-2">Last changed</th>
<th class="col-xs-4">Download</th>
</thead>
<tbody>
<tr>
<td class="col-xs-2">
<i class="far fa-file-alt"></i> <a class="anchor-link" href="#resource-time-series-19-covid-combined" onclick="scrollDown(this)" title="time-series-19-covid-combined">time-series-19-covid-combined</a>
</td>
<td class="col-xs-4" title="">
</td>
<td class="col-xs-1" title="6MB">
                6MB
              </td>
<td class="col-xs-1" title="">
</td>
<td class="download truncate col-xs-4">
<a href="/core/covid-19/r/time-series-19-covid-combined.csv">
                  csv (6MB)
                </a>
                
                ,
                <a href="/core/covid-19/r/time-series-19-covid-combined.json">
                  json (11MB)
                </a>
</td>
</tr>
<tr>
<td class="col-xs

In [4]:
download = res.select('.download')
download

[<td class="download truncate col-xs-4">
 <a href="/core/covid-19/r/time-series-19-covid-combined.csv">
                   csv (6MB)
                 </a>
                 
                 ,
                 <a href="/core/covid-19/r/time-series-19-covid-combined.json">
                   json (11MB)
                 </a>
 </td>,
 <td class="download truncate col-xs-4">
 <a href="/core/covid-19/r/key-countries-pivoted.csv">
                   csv (24kB)
                 </a>
                 
                 ,
                 <a href="/core/covid-19/r/key-countries-pivoted.json">
                   json (44kB)
                 </a>
 </td>,
 <td class="download truncate col-xs-4">
 <a href="/core/covid-19/r/countries-aggregated.csv">
                   csv (2MB)
                 </a>
                 
                 ,
                 <a href="/core/covid-19/r/countries-aggregated.json">
                   json (5MB)
                 </a>
 </td>,
 <td class="download truncate col-x

In [5]:
download.find_all("a")

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [6]:
table.find_all("a")

[<a class="anchor-link" href="#resource-time-series-19-covid-combined" onclick="scrollDown(this)" title="time-series-19-covid-combined">time-series-19-covid-combined</a>,
 <a href="/core/covid-19/r/time-series-19-covid-combined.csv">
                   csv (6MB)
                 </a>,
 <a href="/core/covid-19/r/time-series-19-covid-combined.json">
                   json (11MB)
                 </a>,
 <a class="anchor-link" href="#resource-key-countries-pivoted" onclick="scrollDown(this)" title="key-countries-pivoted">key-countries-pivoted</a>,
 <a href="/core/covid-19/r/key-countries-pivoted.csv">
                   csv (24kB)
                 </a>,
 <a href="/core/covid-19/r/key-countries-pivoted.json">
                   json (44kB)
                 </a>,
 <a class="anchor-link" href="#resource-countries-aggregated" onclick="scrollDown(this)" title="countries-aggregated">countries-aggregated</a>,
 <a href="/core/covid-19/r/countries-aggregated.csv">
                   csv (2MB)
    

The table below is where I want to grab the CVS file from.
Unfortunately it's the second table on the page

In [ ]:
<div class="resource-listing part">
        <table class="table table-condensed table-striped resource-listing">
          <thead>
            <tr><th class="col-xs-2">File</th>
            <th class="col-xs-4">Description</th>
            <th class="col-xs-1">Size</th>
            <th class="col-xs-2">Last changed</th>
            <th class="col-xs-4">Download</th>
          </tr></thead>
          <tbody>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-time-series-19-covid-combined" onclick="scrollDown(this)" class="anchor-link" title="time-series-19-covid-combined">time-series-19-covid-combined</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="6MB">
                6MB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/time-series-19-covid-combined.csv">
                  csv (6MB)
                </a>
                
                ,
                <a href="/core/covid-19/r/time-series-19-covid-combined.json">
                  json (11MB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-key-countries-pivoted" onclick="scrollDown(this)" class="anchor-link" title="key-countries-pivoted">key-countries-pivoted</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="24kB">
                24kB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/key-countries-pivoted.csv">
                  csv (24kB)
                </a>
                
                ,
                <a href="/core/covid-19/r/key-countries-pivoted.json">
                  json (43kB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-countries-aggregated" onclick="scrollDown(this)" class="anchor-link" title="countries-aggregated">countries-aggregated</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="2MB">
                2MB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/countries-aggregated.csv">
                  csv (2MB)
                </a>
                
                ,
                <a href="/core/covid-19/r/countries-aggregated.json">
                  json (5MB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-worldwide-aggregated" onclick="scrollDown(this)" class="anchor-link" title="worldwide-aggregated">worldwide-aggregated</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="20kB">
                20kB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/worldwide-aggregated.csv">
                  csv (20kB)
                </a>
                
                ,
                <a href="/core/covid-19/r/worldwide-aggregated.json">
                  json (34kB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-us_confirmed" onclick="scrollDown(this)" class="anchor-link" title="us_confirmed">us_confirmed</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="145MB">
                145MB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/us_confirmed.csv">
                  csv (145MB)
                </a>
                
                ,
                <a href="/core/covid-19/r/us_confirmed.json">
                  json (249MB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-us_deaths" onclick="scrollDown(this)" class="anchor-link" title="us_deaths">us_deaths</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="152MB">
                152MB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/us_deaths.csv">
                  csv (152MB)
                </a>
                
                ,
                <a href="/core/covid-19/r/us_deaths.json">
                  json (265MB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-reference" onclick="scrollDown(this)" class="anchor-link" title="reference">reference</a>
              </td>
              <td class="col-xs-4" title="">
                
              </td>
              <td class="col-xs-1" title="787kB">
                787kB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/reference.csv">
                  csv (787kB)
                </a>
                
                ,
                <a href="/core/covid-19/r/reference.json">
                  json (1MB)
                </a>
                
              </td>
            </tr>
          
            <tr>
              <td class="col-xs-2">
                <i class="far fa-file-alt"></i> <a href="#resource-covid-19_zip" onclick="scrollDown(this)" class="anchor-link" title="covid-19_zip">covid-19_zip</a>
              </td>
              <td class="col-xs-4" title="Compressed versions of dataset. Includes normalized CSV and JSON data with original data and datapackage.json.">
                Compressed versions of dataset. Includes normalized CSV and JSON data with original data and datapackage.json.
              </td>
              <td class="col-xs-1" title="18MB">
                18MB
              </td>
              <td class="col-xs-1" title="">
                
              </td>
              <td class="download truncate col-xs-4">
                <a href="/core/covid-19/r/covid-19_zip.zip">
                  zip (18MB)
                </a>
                
              </td>
            </tr>
          
          </tbody>
        </table>
      </div>